<a href="https://colab.research.google.com/github/khanhduy0703/desktop-tutorial/blob/main/SONIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import ipywidgets as widgets
from IPython.display import display, HTML
import folium
from geopy.distance import geodesic
import random
import osmnx as ox
import networkx as nx
import numpy as np
import sys
from io import StringIO
from datetime import datetime
import pytz

# CSS cho giao diện đẹp hơn với nền hồng cánh sen chuyển xanh lá và chữ đen
display(HTML("""
<style>
body {
    background: linear-gradient(135deg, #ff9aa2 0%, #ffb7b2 25%, #ffdac1 50%, #e2f0cb 75%, #b5ead7 100%);
    min-height: 100vh;
    margin: 0;
    padding: 0;
}

.vehicle-app {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background: linear-gradient(135deg, #ff9aa2 0%, #ffb7b2 25%, #ffdac1 50%, #e2f0cb 75%, #b5ead7 100%);
    border-radius: 15px;
    padding: 20px;
    margin: 10px 0;
    box-shadow: 0 10px 30px rgba(0,0,0,0.2);
    color: #000000;
}

.app-header {
    color: #000000;
    text-align: center;
    font-size: 28px;
    font-weight: bold;
    margin-bottom: 20px;
    text-shadow: 2px 2px 4px rgba(255,255,255,0.5);
    background: linear-gradient(135deg, #ff9aa2 0%, #ffb7b2 25%, #ffdac1 50%, #e2f0cb 75%, #b5ead7 100%);
    padding: 15px;
    border-radius: 10px;
}

.result-box {
    background: linear-gradient(135deg, rgba(255,154,162,0.3) 0%, rgba(255,183,178,0.3) 25%, rgba(255,218,193,0.3) 50%, rgba(226,240,203,0.3) 75%, rgba(181,234,215,0.3) 100%);
    border-radius: 10px;
    padding: 15px;
    margin: 10px 0;
    box-shadow: 0 5px 15px rgba(0,0,0,0.1);
    color: #000000;
    border: 2px solid rgba(255,255,255,0.4);
}

.vehicle-card {
    background: linear-gradient(135deg, rgba(255,154,162,0.2) 0%, rgba(181,234,215,0.2) 100%);
    border-radius: 10px;
    padding: 12px;
    margin: 8px 0;
    border-left: 4px solid #2d5a27;
    cursor: pointer;
    transition: all 0.3s ease;
    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
    user-select: none;
    color: #000000;
}

.vehicle-card:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 15px rgba(0,0,0,0.2);
    border-left-color: #2d5a27;
    background: linear-gradient(135deg, rgba(226,240,203,0.4) 0%, rgba(181,234,215,0.4) 100%);
}

.vehicle-card.selected {
    border-left-color: #2d5a27;
    background: linear-gradient(135deg, rgba(226,240,203,0.6) 0%, rgba(181,234,215,0.6) 100%);
    transform: translateY(-2px);
    box-shadow: 0 4px 15px rgba(45, 90, 39, 0.3);
}

.vehicle-card.selected::after {
    content: "✅ ĐÃ CHỌN";
    position: absolute;
    top: 10px;
    right: 15px;
    background: #2d5a27;
    color: white;
    padding: 4px 8px;
    border-radius: 12px;
    font-size: 0.8em;
    font-weight: bold;
}

.vehicle-card {
    position: relative;
}

.price-badge {
    background: linear-gradient(45deg, #2d5a27, #4a7c59);
    color: white;
    padding: 6px 12px;
    border-radius: 20px;
    font-weight: bold;
    font-size: 1.1em;
    display: inline-block;
    margin-left: 10px;
    box-shadow: 0 2px 8px rgba(45, 90, 39, 0.3);
}

.surcharge-badge {
    background: linear-gradient(45deg, #d63384, #fd7e14);
    color: white;
    padding: 4px 8px;
    border-radius: 15px;
    font-weight: bold;
    font-size: 0.9em;
    display: inline-block;
    margin-left: 5px;
    box-shadow: 0 2px 6px rgba(214, 51, 132, 0.3);
}

.weather-selector {
    background: linear-gradient(135deg, rgba(255,154,162,0.4) 0%, rgba(181,234,215,0.4) 100%);
    border-radius: 10px;
    padding: 15px;
    margin: 10px 0;
    color: #000000;
    border: 2px solid rgba(255,255,255,0.4);
}

.time-info {
    background: linear-gradient(135deg, rgba(255,154,162,0.5) 0%, rgba(181,234,215,0.5) 100%);
    color: #000000;
    padding: 10px;
    border-radius: 8px;
    margin: 10px 0;
    border: 2px solid rgba(255,255,255,0.4);
}

.vietnam-time-display {
    background: linear-gradient(135deg, rgba(255,215,0,0.3) 0%, rgba(255,140,0,0.3) 100%);
    color: #000000;
    padding: 12px;
    border-radius: 8px;
    margin: 10px 0;
    border: 2px solid rgba(255,165,0,0.6);
    text-align: center;
    font-weight: bold;
}

.click-hint {
    color: #333333;
    font-size: 0.85em;
    font-style: italic;
    margin-top: 5px;
    opacity: 0.8;
}

.success-text { color: #2d5a27; font-weight: bold; }
.info-text { color: #0f4c75; }
.warning-text { color: #b7472a; }
.error-text { color: #8b2635; }

/* Style cho các widget */
.widget-dropdown select {
    background: linear-gradient(135deg, rgba(255,154,162,0.2) 0%, rgba(181,234,215,0.2) 100%) !important;
    color: #000000 !important;
    border: 2px solid rgba(255,255,255,0.4) !important;
}

.widget-text input {
    background: linear-gradient(135deg, rgba(255,154,162,0.2) 0%, rgba(181,234,215,0.2) 100%) !important;
    color: #000000 !important;
    border: 2px solid rgba(255,255,255,0.4) !important;
}

/* Toàn bộ container */
.jp-RenderedHTMLCommon {
    background: linear-gradient(135deg, #ff9aa2 0%, #ffb7b2 25%, #ffdac1 50%, #e2f0cb 75%, #b5ead7 100%);
    color: #000000;
}
</style>
"""))

# Dữ liệu xe mở rộng với 20 mục - Thông tin chi tiết và giá cả đa dạng
vehicles = [
    # Khu vực Quận 1
    {'lat': 10.7721, 'lon': 106.6983, 'type': 'car', 'name': 'Xe 1 - Chợ Bến Thành', 'driver': 'Anh Minh', 'rating': 4.8, 'base_price': 15000, 'per_km': 12000, 'car_type': 'Sedan', 'seats': 4},
    {'lat': 10.7798, 'lon': 106.6950, 'type': 'motorcycle', 'name': 'Xe 2 - Dinh Độc Lập', 'driver': 'Chị Lan', 'rating': 4.9, 'base_price': 8000, 'per_km': 8000, 'car_type': 'Honda Winner', 'seats': 2},
    {'lat': 10.7797, 'lon': 106.6992, 'type': 'car', 'name': 'Xe 3 - Nhà thờ Đức Bà', 'driver': 'Anh Tuấn', 'rating': 4.7, 'base_price': 15000, 'per_km': 12000, 'car_type': 'Sedan', 'seats': 4},
    {'lat': 10.7791, 'lon': 106.6984, 'type': 'motorcycle', 'name': 'Xe 4 - Bưu điện TT', 'driver': 'Anh Nam', 'rating': 4.6, 'base_price': 8000, 'per_km': 8000, 'car_type': 'Yamaha Exciter', 'seats': 2},
    {'lat': 10.7770, 'lon': 106.7000, 'type': 'car', 'name': 'Xe 5 - Phố đi bộ', 'driver': 'Chị Hoa', 'rating': 4.8, 'base_price': 15000, 'per_km': 12000, 'car_type': 'Sedan', 'seats': 4},

    # Khu vực mở rộng Quận 1
    {'lat': 10.7735, 'lon': 106.6988, 'type': 'car', 'name': 'Xe 6 - Công viên 30/4', 'driver': 'Anh Đức', 'rating': 4.9, 'base_price': 18000, 'per_km': 14000, 'car_type': 'SUV', 'seats': 7},
    {'lat': 10.7758, 'lon': 106.7015, 'type': 'motorcycle', 'name': 'Xe 7 - Bitexco', 'driver': 'Chị Mai', 'rating': 4.7, 'base_price': 7500, 'per_km': 7500, 'car_type': 'Honda Vision', 'seats': 2},
    {'lat': 10.7804, 'lon': 106.6945, 'type': 'car', 'name': 'Xe 8 - Opera House', 'driver': 'Anh Khoa', 'rating': 4.6, 'base_price': 16000, 'per_km': 13000, 'car_type': 'Hatchback', 'seats': 5},
    {'lat': 10.7742, 'lon': 106.7025, 'type': 'motorcycle', 'name': 'Xe 9 - Vincom Center', 'driver': 'Anh Tâm', 'rating': 4.8, 'base_price': 8500, 'per_km': 8500, 'car_type': 'Honda Air Blade', 'seats': 2},
    {'lat': 10.7785, 'lon': 106.6975, 'type': 'car', 'name': 'Xe 10 - Diamond Plaza', 'driver': 'Chị Linh', 'rating': 4.5, 'base_price': 17000, 'per_km': 13500, 'car_type': 'Crossover', 'seats': 5},

    # Khu vực Quận 3
    {'lat': 10.7820, 'lon': 106.6920, 'type': 'motorcycle', 'name': 'Xe 11 - Công viên Tao Đàn', 'driver': 'Anh Hưng', 'rating': 4.4, 'base_price': 7000, 'per_km': 7000, 'car_type': 'Yamaha Sirius', 'seats': 2},
    {'lat': 10.7855, 'lon': 106.6895, 'type': 'car', 'name': 'Xe 12 - Bệnh viện Chợ Rẫy', 'driver': 'Chị Nga', 'rating': 4.9, 'base_price': 16500, 'per_km': 12500, 'car_type': 'Sedan', 'seats': 4},
    {'lat': 10.7875, 'lon': 106.6935, 'type': 'motorcycle', 'name': 'Xe 13 - Trường ĐH Y Dược', 'driver': 'Anh Phong', 'rating': 4.7, 'base_price': 8000, 'per_km': 8000, 'car_type': 'Honda SH', 'seats': 2},
    {'lat': 10.7840, 'lon': 106.6960, 'type': 'car', 'name': 'Xe 14 - Chùa Vĩnh Nghiêm', 'driver': 'Anh Long', 'rating': 4.6, 'base_price': 19000, 'per_km': 15000, 'car_type': 'MPV', 'seats': 7},

    # Khu vực biên giới Quận 1-3
    {'lat': 10.7810, 'lon': 106.6980, 'type': 'motorcycle', 'name': 'Xe 15 - Lê Lợi Intersection', 'driver': 'Chị Thu', 'rating': 4.8, 'base_price': 7800, 'per_km': 7800, 'car_type': 'Honda Lead', 'seats': 2},
    {'lat': 10.7790, 'lon': 106.6955, 'type': 'car', 'name': 'Xe 16 - Tòa nhà Saigon Centre', 'driver': 'Anh Việt', 'rating': 4.5, 'base_price': 15500, 'per_km': 12800, 'car_type': 'Sedan', 'seats': 4},
    {'lat': 10.7765, 'lon': 106.6940, 'type': 'motorcycle', 'name': 'Xe 17 - Trần Hưng Đạo', 'driver': 'Anh Quang', 'rating': 4.9, 'base_price': 8200, 'per_km': 8200, 'car_type': 'Yamaha NVX', 'seats': 2},
    {'lat': 10.7825, 'lon': 106.7005, 'type': 'car', 'name': 'Xe 18 - Landmark 81 View', 'driver': 'Chị Huyền', 'rating': 4.7, 'base_price': 20000, 'per_km': 16000, 'car_type': 'Luxury Sedan', 'seats': 4},

    # Khu vực xa hơn
    {'lat': 10.7680, 'lon': 106.6900, 'type': 'motorcycle', 'name': 'Xe 19 - Khu Chợ Lớn', 'driver': 'Anh Hải', 'rating': 4.3, 'base_price': 6500, 'per_km': 6500, 'car_type': 'Honda Wave', 'seats': 2},
    {'lat': 10.7900, 'lon': 106.7050, 'type': 'car', 'name': 'Xe 20 - Vinhomes Central Park', 'driver': 'Chị Thảo', 'rating': 4.9, 'base_price': 22000, 'per_km': 17000, 'car_type': 'Premium SUV', 'seats': 7}
]

# ============== HÀM THỜI GIAN VIỆT NAM ================
def get_vietnam_time():
    """Lấy thời gian hiện tại tại Việt Nam (GMT+7)"""
    vietnam_tz = pytz.timezone('Asia/Ho_Chi_Minh')
    vietnam_time = datetime.now(vietnam_tz)
    return vietnam_time

def is_night_time():
    """Kiểm tra có phải giờ ban đêm không (22h-6h) theo giờ Việt Nam"""
    vietnam_time = get_vietnam_time()
    current_hour = vietnam_time.hour
    return current_hour >= 22 or current_hour < 6

def get_time_period():
    """Xác định khung thời gian trong ngày theo giờ Việt Nam"""
    vietnam_time = get_vietnam_time()
    current_hour = vietnam_time.hour

    if 6 <= current_hour < 12:
        return "🌅 Sáng"
    elif 12 <= current_hour < 18:
        return "☀️ Chiều"
    elif 18 <= current_hour < 22:
        return "🌆 Tối"
    else:  # 22h-6h
        return "🌙 Đêm"

def format_vietnam_time():
    """Format thời gian Việt Nam để hiển thị"""
    vietnam_time = get_vietnam_time()
    weekdays = ['Thứ 2', 'Thứ 3', 'Thứ 4', 'Thứ 5', 'Thứ 6', 'Thứ 7', 'Chủ nhật']
    weekday = weekdays[vietnam_time.weekday()]

    date_str = vietnam_time.strftime(f"{weekday}, %d/%m/%Y")
    time_str = vietnam_time.strftime("%H:%M:%S")

    return f"{date_str} - {time_str}"

# Widget inputs với style đẹp hơn và thêm tính năng mới
vehicle_type = widgets.Dropdown(
    options=['🚗 Tất cả', '🚙 Ô tô', '🏍️ Xe máy'],
    value='🚗 Tất cả',
    description='Loại xe:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

weather_selector = widgets.Dropdown(
    options=['☀️ Nắng', '🌧️ Mưa'],
    value='☀️ Nắng',
    description='Thời tiết:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

lat_input = widgets.FloatText(
    value=10.775,
    description='📍 Vĩ độ:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

lon_input = widgets.FloatText(
    value=106.700,
    description='📍 Kinh độ:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

find_btn = widgets.Button(
    description='🔍 Tìm xe khả dụng',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px'),
    style={'button_color': '#27ae60', 'font_weight': 'bold'}
)

# Widget hiển thị thời gian Việt Nam
vietnam_time_widget = widgets.HTML()

def update_vietnam_time_display():
    """Cập nhật hiển thị thời gian Việt Nam"""
    vietnam_time_str = format_vietnam_time()
    time_period = get_time_period()
    is_night = is_night_time()

    night_status = "🌙 PHỤ PHÍ BAN ĐÊM (+20%)" if is_night else "☀️ KHÔNG PHỤ PHÍ"

    vietnam_time_widget.value = f"""
    <div class="vietnam-time-display">
        🇻🇳 THỜI GIAN VIỆT NAM: {vietnam_time_str}<br>
        🕐 Khung giờ: {time_period} | {night_status}
    </div>
    """

# Cập nhật thời gian lần đầu
update_vietnam_time_display()

result_output = widgets.Output()
vehicle_selection_output = widgets.Output()
map_output = widgets.Output()

# Biến global để lưu thông tin xe được chọn
selected_vehicle = None
available_vehicles = []

def get_time_surcharge():
    """Tính phụ phí theo giờ Việt Nam"""
    return 0.2 if is_night_time() else 0.0

def get_weather_surcharge():
    """Tính phụ phí theo thời tiết"""
    return 0.2 if '🌧️' in weather_selector.value else 0.0

def calculate_price(distance_km, vehicle):
    """Tính giá cước dựa trên khoảng cách, giờ Việt Nam và thời tiết"""
    base_price = vehicle['base_price']
    per_km_price = vehicle['per_km']

    # Tính giá cơ bản
    basic_price = base_price + (distance_km * per_km_price)

    # Tính các phụ phí
    time_surcharge = get_time_surcharge()
    weather_surcharge = get_weather_surcharge()
    total_surcharge = time_surcharge + weather_surcharge

    # Áp dụng phụ phí
    surcharge_amount = basic_price * total_surcharge
    total_price = basic_price + surcharge_amount

    # Làm tròn theo 500đ
    total_price = round(total_price / 500) * 500

    return {
        'basic_price': int(basic_price),
        'total_price': int(total_price),
        'time_surcharge': time_surcharge,
        'weather_surcharge': weather_surcharge,
        'surcharge_amount': int(surcharge_amount)
    }

def get_current_time_info():
    """Lấy thông tin thời gian hiện tại theo giờ Việt Nam"""
    vietnam_time = get_vietnam_time()
    current_time = vietnam_time.strftime("%H:%M:%S")
    time_period = get_time_period()
    is_night = is_night_time()

    surcharge_info = "(+20% phụ phí)" if is_night else "(Không phụ phí)"

    return f"{time_period} - {current_time} {surcharge_info}"

def format_currency(amount):
    """Format tiền tệ VND"""
    return f"{amount:,}đ".replace(',', '.')

def get_price_breakdown_html(price_info, distance_km):
    """Tạo HTML hiển thị chi tiết giá"""
    html = f"""
    <div style="background: linear-gradient(135deg, rgba(255,154,162,0.2) 0%, rgba(181,234,215,0.2) 100%); padding: 10px; border-radius: 5px; margin: 5px 0; color: #000000; border: 2px solid rgba(255,255,255,0.4);">
        <strong>💰 Chi tiết giá cước (theo giờ Việt Nam):</strong><br>
        • Giá cơ bản: <strong>{format_currency(price_info['basic_price'])}</strong><br>
    """

    if price_info['time_surcharge'] > 0:
        html += f"• Phụ phí ban đêm (22h-6h VN) (+{price_info['time_surcharge']*100:.0f}%): <strong style='color: #b7472a;'>+{format_currency(int(price_info['basic_price'] * price_info['time_surcharge']))}</strong><br>"

    if price_info['weather_surcharge'] > 0:
        html += f"• Phụ phí thời tiết (+{price_info['weather_surcharge']*100:.0f}%): <strong style='color: #0f4c75;'>+{format_currency(int(price_info['basic_price'] * price_info['weather_surcharge']))}</strong><br>"

    if price_info['surcharge_amount'] > 0:
        html += f"• <strong>Tổng phụ phí: <span style='color: #8b2635;'>+{format_currency(price_info['surcharge_amount'])}</span></strong><br>"

    html += f"• <strong>THÀNH TIỀN: <span style='color: #2d5a27; font-size: 1.2em;'>{format_currency(price_info['total_price'])}</span></strong>"
    html += "</div>"

    return html

def suppress_print(func):
    """Decorator để ẩn các print statement"""
    def wrapper(*args, **kwargs):
        # Capture stdout
        old_stdout = sys.stdout
        sys.stdout = captured_output = StringIO()

        try:
            result = func(*args, **kwargs)
        finally:
            # Restore stdout
            sys.stdout = old_stdout

        return result
    return wrapper

@suppress_print
def get_real_route(start_lat, start_lon, end_lat, end_lon):
    """
    Tìm tuyến đường thực tế giữa 2 điểm trên đường phố (im lặng)
    """
    try:
        # Tính khoảng cách để xác định phạm vi tải dữ liệu
        straight_distance = geodesic((start_lat, start_lon), (end_lat, end_lon)).kilometers

        # Tạo bounding box rộng hơn để đảm bảo có đủ đường
        if straight_distance < 1:
            padding = 0.01
        elif straight_distance < 3:
            padding = 0.02
        else:
            padding = straight_distance * 0.3

        north = max(start_lat, end_lat) + padding
        south = min(start_lat, end_lat) - padding
        east = max(start_lon, end_lon) + padding
        west = min(start_lon, end_lon) - padding

        # Thử các phương thức tải graph khác nhau
        G = None

        # Method 1: Tải từ bounding box với drive network
        try:
            G = ox.graph_from_bbox(north, south, east, west, network_type='drive', simplify=True)
            if G and len(G.nodes()) > 10:
                pass
            else:
                G = None
        except:
            pass

        # Method 2: Tải từ trung tâm với khoảng cách
        if G is None:
            try:
                center_lat = (start_lat + end_lat) / 2
                center_lon = (start_lon + end_lon) / 2
                radius = max(straight_distance * 1000 * 1.5, 2000)

                G = ox.graph_from_point((center_lat, center_lon), dist=radius, network_type='drive', simplify=True)
                if G and len(G.nodes()) > 10:
                    pass
                else:
                    G = None
            except:
                pass

        if G is None or len(G.nodes()) < 5:
            raise Exception("Không thể tải đủ dữ liệu đường")

        # Tìm node gần nhất
        try:
            start_node = ox.distance.nearest_nodes(G, start_lon, start_lat)
            end_node = ox.distance.nearest_nodes(G, end_lon, end_lat)
        except:
            # Fallback: tìm node manually
            min_start_dist = float('inf')
            min_end_dist = float('inf')
            start_node = end_node = None

            for node, data in G.nodes(data=True):
                node_lat, node_lon = data['y'], data['x']

                start_dist = geodesic((start_lat, start_lon), (node_lat, node_lon)).meters
                if start_dist < min_start_dist:
                    min_start_dist = start_dist
                    start_node = node

                end_dist = geodesic((end_lat, end_lon), (node_lat, node_lon)).meters
                if end_dist < min_end_dist:
                    min_end_dist = end_dist
                    end_node = node

            if start_node is None or end_node is None:
                raise Exception("Không tìm thấy node phù hợp")

        # Tìm đường đi
        route = None
        weights_to_try = ['length', 'travel_time', None]

        for weight in weights_to_try:
            try:
                if nx.has_path(G, start_node, end_node):
                    if weight:
                        route = nx.shortest_path(G, start_node, end_node, weight=weight)
                    else:
                        route = nx.shortest_path(G, start_node, end_node)
                    break
            except:
                continue

        if route is None:
            # Thử tìm đường gián tiếp
            try:
                mid_lat = (start_lat + end_lat) / 2
                mid_lon = (start_lon + end_lon) / 2
                mid_node = ox.distance.nearest_nodes(G, mid_lon, mid_lat)

                route1 = nx.shortest_path(G, start_node, mid_node)
                route2 = nx.shortest_path(G, mid_node, end_node)
                route = route1 + route2[1:]
            except:
                raise Exception("Không thể tìm đường đi")

        # Chuyển đổi route thành coordinates
        route_coords = []
        total_distance = 0

        for i, node in enumerate(route):
            node_data = G.nodes[node]
            lat, lon = node_data['y'], node_data['x']
            route_coords.append([lat, lon])

            if i > 0:
                prev_lat, prev_lon = route_coords[i-1]
                segment_dist = geodesic((prev_lat, prev_lon), (lat, lon)).meters
                total_distance += segment_dist

        # Đảm bảo route bắt đầu và kết thúc đúng vị trí
        if len(route_coords) > 0:
            start_to_first = geodesic((start_lat, start_lon), route_coords[0]).meters
            if start_to_first > 20:
                route_coords.insert(0, [start_lat, start_lon])
                total_distance += start_to_first

            last_to_end = geodesic(route_coords[-1], (end_lat, end_lon)).meters
            if last_to_end > 20:
                route_coords.append([end_lat, end_lon])
                total_distance += last_to_end

        if len(route_coords) < 2:
            raise Exception("Route quá ngắn")

        return route_coords, total_distance

    except:
        # Enhanced fallback
        try:
            num_intermediate = max(3, int(geodesic((start_lat, start_lon), (end_lat, end_lon)).kilometers * 2))

            intermediate_coords = []
            for i in range(num_intermediate + 1):
                ratio = i / num_intermediate
                lat = start_lat + (end_lat - start_lat) * ratio
                lon = start_lon + (end_lon - start_lon) * ratio

                if i > 0 and i < num_intermediate:
                    offset = 0.0002 * np.sin(ratio * np.pi)
                    lat += offset
                    lon += offset

                intermediate_coords.append([lat, lon])

            straight_dist = geodesic((start_lat, start_lon), (end_lat, end_lon)).meters
            estimated_distance = straight_dist * 1.3

            return intermediate_coords, estimated_distance

        except:
            straight_coords = [[start_lat, start_lon], [end_lat, end_lon]]
            straight_distance = geodesic((start_lat, start_lon), (end_lat, end_lon)).meters
            return straight_coords, straight_distance

def on_vehicle_select(button):
    """Xử lý khi người dùng chọn xe"""
    global selected_vehicle
    selected_vehicle = button.vehicle_data

    # Cập nhật hiển thị thời gian Việt Nam
    update_vietnam_time_display()

    # Cập nhật giao diện chọn xe
    update_vehicle_selection_ui()

    # Hiển thị bản đồ với xe được chọn
    show_map_with_selected_vehicle()

def update_vehicle_selection_ui():
    """Cập nhật giao diện danh sách xe"""
    with vehicle_selection_output:
        vehicle_selection_output.clear_output()

        # Hiển thị thời gian Việt Nam
        display(vietnam_time_widget)

        # Hiển thị thông tin thời gian và thời tiết hiện tại
        time_info = get_current_time_info()
        weather_info = weather_selector.value

        display(HTML(f"""
        <div class="time-info">
            <h4 style="margin-top: 0; color: #000000;">⏰ Thông tin đặt xe</h4>
            <strong style="color: #000000;">🕐 Khung giờ:</strong> <span style="color: #000000;">{time_info}</span><br>
            <strong style="color: #000000;">🌤️ Thời tiết:</strong> <span style="color: #000000;">{weather_info} {'(+20% phụ phí)' if '🌧️' in weather_info else '(Không phụ phí)'}</span>
        </div>
        """))

        display(HTML("""
        <div class="result-box">
            <h3 style="color: #2c3e50; margin-top: 0;">🚗 Chọn xe từ danh sách dưới đây:</h3>
            <p style="color: #7f8c8d; margin: 5px 0;"><i>💡 Giá đã bao gồm phụ phí (nếu có) - Click trực tiếp vào card để chọn xe</i></p>
        </div>
        """))

        # Tạo container cho các xe với button trong suốt phủ lên card
        for i, vehicle in enumerate(available_vehicles):
            user_pos = (lat_input.value, lon_input.value)
            vehicle_pos = (vehicle['lat'], vehicle['lon'])
            distance = geodesic(user_pos, vehicle_pos).meters
            distance_km = distance / 1000

            # Tính giá với phụ phí
            price_info = calculate_price(distance_km, vehicle)

            # Icon và màu theo loại xe
            icon = "🚙" if vehicle['type'] == 'car' else "🏍️"

            # Kiểm tra xe có được chọn không
            is_selected = selected_vehicle and selected_vehicle['name'] == vehicle['name']

            # Tạo phần hiển thị phụ phí
            surcharge_badges = ""
            if price_info['time_surcharge'] > 0:
                surcharge_badges += '<span class="surcharge-badge">🌙 +20%</span>'
            if price_info['weather_surcharge'] > 0:
                surcharge_badges += '<span class="surcharge-badge">🌧️ +20%</span>'

            # HTML thông tin xe
            vehicle_info_html = f"""
            <div class="vehicle-card {'selected' if is_selected else ''}" style="position: relative; margin-bottom: 5px;">
                <div style="display: flex; justify-content: space-between; align-items: center;">
                    <div style="flex: 1;">
                        <strong style="font-size: 1.1em;">{icon} {vehicle['name']}</strong>{surcharge_badges}<br>
                        <div style="margin: 8px 0; line-height: 1.4;">
                            <span style="color: #495057;">👤 <strong>{vehicle['driver']}</strong></span> |
                            <span style="color: #ffc107;">⭐ {vehicle['rating']}/5.0</span> |
                            <span style="color: #6c757d;">📏 {distance:.0f}m</span><br>
                            <span style="color: #007bff;">🚗 {vehicle['car_type']} ({vehicle['seats']} chỗ)</span> |
                            <span style="color: #28a745;">💰 {vehicle['base_price']:,}đ + {vehicle['per_km']:,}đ/km</span>
                        </div>
                        {' <strong style="color: #28a745;">✓ ĐÃ CHỌN</strong>' if is_selected else ' <span style="color: #007bff;">👆 Click để chọn</span>'}
                    </div>
                    <div>
                        <span class="price-badge">{format_currency(price_info['total_price'])}</span>
                    </div>
                </div>
            </div>
            """

            display(HTML(vehicle_info_html))

            # Tạo button chọn xe với style đẹp
            select_btn = widgets.Button(
                description=f"{'✅ ĐÃ CHỌN' if is_selected else '🔘 CHỌN'} {icon} {vehicle['name']} - {format_currency(price_info['total_price'])}",
                button_style='success' if is_selected else 'info',
                layout=widgets.Layout(
                    width='100%',
                    height='45px',
                    margin='0px 0 8px 0'
                ),
                style={
                    'button_color': '#28a745' if is_selected else '#17a2b8',
                    'font_weight': 'bold'
                }
            )

            # Gán dữ liệu xe vào button
            select_btn.vehicle_data = vehicle
            select_btn.on_click(on_vehicle_select)

            display(select_btn)

def show_map_with_selected_vehicle():
    """Hiển thị bản đồ với xe được chọn"""
    if not selected_vehicle:
        with map_output:
            map_output.clear_output()
            display(HTML('<div class="result-box"><div class="warning-text">⚠️ Vui lòng chọn xe trước!</div></div>'))
        return

    with map_output:
        map_output.clear_output()

        # Thông báo đang xử lý
        display(HTML(f"""
        <div class="result-box">
            <div class="success-text">✅ Xe đã được chọn: {selected_vehicle['name']}</div>
            <div class="info-text">🗺️ Đang tạo bản đồ và tính toán tuyến đường...</div>
        </div>
        """))

        try:
            # Tính toán thông tin cơ bản
            user_pos = (lat_input.value, lon_input.value)
            vehicle_pos = (selected_vehicle['lat'], selected_vehicle['lon'])
            distance = geodesic(user_pos, vehicle_pos).meters
            distance_km = distance / 1000
            price_info = calculate_price(distance_km, selected_vehicle)

            # Hiển thị thông tin xe đã chọn với chi tiết mở rộng
            time_info = get_current_time_info()
            weather_info = weather_selector.value
            vietnam_time_str = format_vietnam_time()

            display(HTML(f"""
            <div class="result-box">
                <strong>🚗 Thông tin xe đã chọn:</strong><br>
                👤 Tài xế: <strong>{selected_vehicle['driver']}</strong><br>
                ⭐ Đánh giá: <strong>{selected_vehicle['rating']}/5.0</strong><br>
                🚦 Loại xe: <strong>{selected_vehicle['car_type']} ({selected_vehicle['seats']} chỗ ngồi)</strong><br>
                📏 Khoảng cách chim bay: <strong>{distance:.0f}m</strong><br>
                💰 Cấu trúc giá: <strong>{selected_vehicle['base_price']:,}đ cố định + {selected_vehicle['per_km']:,}đ/km</strong><br>
                🇻🇳 Thời gian VN: <strong>{vietnam_time_str}</strong><br>
                🕐 Khung giờ: <strong>{time_info}</strong><br>
                🌤️ Thời tiết: <strong>{weather_info}</strong>
            </div>
            """))

            # Hiển thị chi tiết giá cước
            display(HTML(get_price_breakdown_html(price_info, distance_km)))

            # Create map
            map_center = [(lat_input.value + selected_vehicle['lat'])/2,
                         (lon_input.value + selected_vehicle['lon'])/2]

            m = folium.Map(
                location=map_center,
                zoom_start=15,
                tiles='CartoDB positron'
            )

            # User position marker
            folium.Marker(
                [lat_input.value, lon_input.value],
                popup=folium.Popup("""
                <div style='font-family: Arial; text-align: center; min-width: 150px;'>
                    <b>📍 Vị trí của bạn</b><br>
                    <i>Điểm xuất phát</i>
                </div>
                """, max_width=200),
                icon=folium.Icon(color='green', icon='user', prefix='fa'),
                tooltip="Vị trí của bạn"
            ).add_to(m)

            # Selected vehicle marker
            icon_name = 'car' if selected_vehicle['type'] == 'car' else 'motorcycle'

            # Tạo popup cho xe đã chọn với thông tin chi tiết giá
            selected_popup_content = f"""
            <div style='font-family: Arial; min-width: 350px; text-align: center;'>
                <h4 style='margin-top: 0; color: #e74c3c;'>⭐ XE ĐÃ CHỌN ⭐</h4>
                <h5 style='color: #2c3e50;'>{selected_vehicle['name']}</h5>
                <p><b>👤 Tài xế:</b> {selected_vehicle['driver']}</p>
                <p><b>⭐ Đánh giá:</b> {selected_vehicle['rating']}/5.0</p>
                <p><b>🚗 Xe:</b> {selected_vehicle['car_type']} ({selected_vehicle['seats']} chỗ)</p>
                <p><b>📏 Khoảng cách:</b> {distance:.0f}m</p>
                <p><b>🇻🇳 Thời gian VN:</b> {vietnam_time_str}</p>
                <hr style='margin: 10px 0;'>
                <p><b>💰 Giá cơ bản:</b> {format_currency(price_info['basic_price'])}</p>
            """

            if price_info['surcharge_amount'] > 0:
                selected_popup_content += f"""
                <p><b>💸 Phụ phí:</b> <span style='color: #e74c3c;'>+{format_currency(price_info['surcharge_amount'])}</span></p>
                """

            selected_popup_content += f"""
                <p><b>🏆 TỔNG TIỀN:</b> <span style='color: #28a745; font-weight: bold; font-size: 1.2em;'>{format_currency(price_info['total_price'])}</span></p>
            </div>
            """

            folium.Marker(
                [selected_vehicle['lat'], selected_vehicle['lon']],
                popup=folium.Popup(selected_popup_content, max_width=400),
                icon=folium.Icon(color='orange', icon=icon_name, prefix='fa'),
                tooltip=f"⭐ {selected_vehicle['name']} - {format_currency(price_info['total_price'])}"
            ).add_to(m)

            # Display other available vehicles with lower opacity
            for vehicle in available_vehicles:
                if vehicle['name'] != selected_vehicle['name']:
                    color = 'blue' if vehicle['type'] == 'car' else 'red'
                    icon = 'car' if vehicle['type'] == 'car' else 'motorcycle'

                    v_distance = geodesic(user_pos, (vehicle['lat'], vehicle['lon'])).meters
                    v_distance_km = v_distance / 1000
                    v_price_info = calculate_price(v_distance_km, vehicle)

                    other_popup_content = f"""
                    <div style='font-family: Arial; min-width: 250px;'>
                        <h4 style='margin-top: 0; color: #6c757d;'>{vehicle['name']}</h4>
                        <p><b>👤 Tài xế:</b> {vehicle['driver']}</p>
                        <p><b>⭐ Đánh giá:</b> {vehicle['rating']}/5.0</p>
                        <p><b>📏 Khoảng cách:</b> {v_distance:.0f}m</p>
                        <p><b>💰 Giá cơ bản:</b> {format_currency(v_price_info['basic_price'])}</p>
                    """

                    if v_price_info['surcharge_amount'] > 0:
                        other_popup_content += f"<p><b>💸 Phụ phí:</b> +{format_currency(v_price_info['surcharge_amount'])}</p>"

                    other_popup_content += f"""
                        <p><b>🏆 TỔNG:</b> <span style='color: #28a745; font-weight: bold;'>{format_currency(v_price_info['total_price'])}</span></p>
                        <p><small><i>Bấm để chọn xe này</i></small></p>
                    </div>
                    """

                    folium.Marker(
                        [vehicle['lat'], vehicle['lon']],
                        popup=folium.Popup(other_popup_content, max_width=300),
                        icon=folium.Icon(color=color, icon=icon, prefix='fa'),
                        tooltip=f"{vehicle['name']} - {format_currency(v_price_info['total_price'])}"
                    ).add_to(m)

            # Calculate and draw the route
            print("🛣️ Đang tính toán tuyến đường...")

            route_coords, route_distance = get_real_route(
                lat_input.value, lon_input.value,
                selected_vehicle['lat'], selected_vehicle['lon']
            )

            if route_coords and len(route_coords) >= 2:
                is_real_route = len(route_coords) > 2

                if is_real_route:
                    line_color = '#28a745'
                    route_type = "🛣️ Tuyến đường thực tế"
                    dash_array = None
                else:
                    line_color = '#ffc107'
                    route_type = "📏 Đường ước lượng"
                    dash_array = '10,5'

                # Calculate final price based on route distance
                route_distance_km = route_distance / 1000
                final_price_info = calculate_price(route_distance_km, selected_vehicle)

                # Draw the route
                folium.PolyLine(
                    route_coords,
                    color=line_color,
                    weight=5,
                    opacity=0.8,
                    dash_array=dash_array,
                    popup=folium.Popup(f"""
                    <div style='font-family: Arial; text-align: center;'>
                        <b>{route_type}</b><br>
                        📏 Khoảng cách: <b>{route_distance:.0f}m</b><br>
                        ⏱️ Thời gian ước tính: <b>{route_distance/250:.1f} phút</b><br>
                        💰 Giá cơ bản: <b>{format_currency(final_price_info['basic_price'])}</b><br>
                        💸 Phụ phí: <b>{format_currency(final_price_info['surcharge_amount'])}</b><br>
                        🏆 Tổng tiền: <b style='color: #28a745;'>{format_currency(final_price_info['total_price'])}</b>
                    </div>
                    """, max_width=300)
                ).add_to(m)

                # Start point marker
                folium.CircleMarker(
                    route_coords[0],
                    radius=8,
                    popup="🚀 Điểm xuất phát",
                    color='green',
                    fillColor='lightgreen',
                    fillOpacity=0.8,
                    weight=3
                ).add_to(m)

                # End point marker
                folium.CircleMarker(
                    route_coords[-1],
                    radius=8,
                    popup="🏁 Điểm đến",
                    color='orange',
                    fillColor='yellow',
                    fillOpacity=0.8,
                    weight=3
                ).add_to(m)

                # Display detailed info with route-based pricing
                display(HTML(f"""
                <div class="result-box">
                    <div class="success-text">🛣️ Thông tin chuyến đi chi tiết:</div>
                    📏 Khoảng cách theo đường: <strong>{route_distance:.0f}m ({route_distance_km:.2f}km)</strong><br>
                    ⏱️ Thời gian di chuyển ước tính: <strong>{route_distance/250:.1f} phút</strong><br>
                    🎯 Loại tuyến đường: <strong>{'Đường thực tế theo OSM' if len(route_coords) > 2 else 'Đường ước lượng'}</strong><br>
                    📱 <strong style='color: #007bff;'>Sẵn sàng đặt xe!</strong>
                </div>
                """))

                # Hiển thị chi tiết giá cuối cùng
                display(HTML(get_price_breakdown_html(final_price_info, route_distance_km)))

            else:
                display(HTML(f"""
                <div class="result-box">
                    <div class="warning-text">⚠️ Không thể tính toán tuyến đường chi tiết</div>
                </div>
                """))
                display(HTML(get_price_breakdown_html(price_info, distance_km)))

            # Fit map bounds to show all points
            all_coords = [
                [lat_input.value, lon_input.value],
                [selected_vehicle['lat'], selected_vehicle['lon']]
            ]
            m.fit_bounds(all_coords)

            # Display the map
            display(m)

            # Legend with updated info
            display(HTML(f"""
            <div class="result-box">
                <div class="info-text">
                    <strong>🎯 Chú thích bản đồ:</strong><br>
                    • 🟢 Marker xanh: Vị trí của bạn<br>
                    • 🟠 Marker cam: Xe đã chọn<br>
                    • 🔵🔴 Marker khác: Các xe khác (có thể click để chọn)<br>
                    • ━━━ Đường xanh lá: Tuyến đường thực tế theo OSM<br>
                    • ┈┈┈ Đường vàng đứt nét: Đường ước lượng<br>
                    • 🚀🏁 Điểm xanh/cam: Điểm xuất phát/đích<br><br>
                    <strong>💰 Phụ phí áp dụng (theo giờ Việt Nam):</strong><br>
                    • 🌙 Ban đêm (22h-6h VN): +20%<br>
                    • 🌧️ Thời tiết mưa: +20%<br>
                    • 📊 Các phụ phí có thể cộng dồn<br>
                    • 🇻🇳 Thời gian hiện tại: <strong>{vietnam_time_str}</strong>
                </div>
            </div>
            """))

        except Exception as e:
            display(HTML(f"""
            <div class="result-box">
                <div class="error-text">❌ Lỗi hiển thị bản đồ: {str(e)}</div>
                <div class="info-text">💡 Vui lòng thử lại hoặc chọn xe khác.</div>
            </div>
            """))
            print(f"Lỗi chi tiết: {e}")

def find_available_vehicles(b):
    global available_vehicles
    global selected_vehicle # Reset selected vehicle when searching again
    selected_vehicle = None

    # Cập nhật thời gian Việt Nam
    update_vietnam_time_display()

    with result_output:
        result_output.clear_output()

        display(HTML("""
        <div class="result-box">
            <h3 style="color: #2c3e50; margin-top: 0;">🔍 Đang tìm kiếm xe khả dụng...</h3>
        </div>
        """))

        # Lọc xe theo loại
        type_filter = vehicle_type.value
        if '🚙 Ô tô' in type_filter:
            filtered = [v for v in vehicles if v['type'] == 'car']
        elif '🏍️ Xe máy' in type_filter:
            filtered = [v for v in vehicles if v['type'] == 'motorcycle']
        else:
            filtered = vehicles

        if not filtered:
            display(HTML('<div class="result-box"><div class="error-text">❌ Không tìm thấy xe phù hợp!</div></div>'))
            return

        # Tính khoảng cách và sắp xếp
        user_pos = (lat_input.value, lon_input.value)
        vehicle_distances = []

        for vehicle in filtered:
            vehicle_pos = (vehicle['lat'], vehicle['lon'])
            dist = geodesic(user_pos, vehicle_pos).meters
            vehicle_distances.append((vehicle, dist))

        # Sort by distance
        vehicle_distances.sort(key=lambda x: x[1])
        available_vehicles = [v[0] for v in vehicle_distances]

        # Hiển thị thông tin điều kiện tìm kiếm
        time_info = get_current_time_info()
        weather_info = weather_selector.value
        vietnam_time_str = format_vietnam_time()

        display(HTML(f"""
        <div class="result-box">
            <div class="success-text">✅ Tìm thấy {len(available_vehicles)} xe khả dụng trong khu vực!</div>
            <div style="margin: 10px 0; padding: 10px; background: #f8f9fa; border-radius: 5px;">
                <strong>🔍 Điều kiện tìm kiếm:</strong><br>
                🇻🇳 Thời gian VN: {vietnam_time_str}<br>
                🕐 Khung giờ: {time_info}<br>
                🌤️ Thời tiết: {weather_info}<br>
                🚗 Loại xe: {type_filter}
            </div>
            <div class="info-text">💡 Giá hiển thị đã bao gồm tất cả phụ phí theo giờ Việt Nam. Bấm vào xe bất kỳ để chọn.</div>
        </div>
        """))

    # Update the vehicle list UI
    update_vehicle_selection_ui()

    # Clear the old map
    with map_output:
        map_output.clear_output()

find_btn.on_click(find_available_vehicles)

print("=== 🚀 KHỞI ĐỘNG ỨNG DỤNG ĐẶT XE THÔNG MINH VỚI THỜI GIAN VIỆT NAM ===")

# Main UI layout
header = widgets.HTML("""
<div class="app-header">
    🇻🇳 ỨNG DỤNG ĐẶT XE THÔNG MINH 🚗🏍️<br>
    <small style="font-size: 0.6em; color: #34495e;">Với tính năng phụ phí theo giờ Việt Nam và thời tiết</small>
</div>
""")

# Input section - thêm weather selector
input_section = widgets.HBox([
    vehicle_type, weather_selector, lat_input, lon_input, find_btn
], layout=widgets.Layout(justify_content='center', align_items='center'))

# Instructions with updated info
instructions = widgets.HTML(f"""
<div class="result-box">
    <h3 style="color: #2c3e50; margin-top: 0;">📋 Hướng dẫn sử dụng</h3>
    <ol style="line-height: 1.6;">
        <li>🚗 Chọn loại xe muốn tìm (Tất cả/Ô tô/Xe máy)</li>
        <li>🌤️ <strong>Chọn điều kiện thời tiết hiện tại</strong></li>
        <li>📍 Nhập tọa độ vị trí hiện tại của bạn</li>
        <li>🔍 Nhấn nút "Tìm xe khả dụng"</li>
        <li>🚗 Chọn xe từ danh sách hiển thị</li>
        <li>🗺️ Xem bản đồ chi tiết và thông tin giá cước</li>
    </ol>

    <h4 style="color: #27ae60;">🌟 Tính năng mới:</h4>
    <ul style="line-height: 1.6;">
        <li>✅ Chọn xe tùy ý từ danh sách</li>
        <li>💰 Hiển thị giá cước theo khoảng cách</li>
        <li>🛣️ Tìm tuyến đường thực tế (im lặng)</li>
        <li>⭐ So sánh giá và đánh giá tài xế</li>
        <li>🗺️ Bản đồ tương tác với thông tin đầy đủ</li>
        <li>🇻🇳 <strong>THỜI GIAN VIỆT NAM CHÍNH XÁC (GMT+7)</strong></li>
        <li>🌙 <strong>Phụ phí ban đêm (22h-6h VN): +20%</strong></li>
        <li>🌧️ <strong>Phụ phí thời tiết mưa: +20%</strong></li>
        <li>📊 <strong>Các phụ phí có thể cộng dồn</strong></li>
    </ul>

    <h4 style="color: #3498db;">💰 Bảng giá tham khảo:</h4>
    <ul style="line-height: 1.6;">
        <li>🚙 <strong>Ô tô:</strong> Giá khởi điểm 15.000đ + 12.000đ/km</li>
        <li>🏍️ <strong>Xe máy:</strong> Giá khởi điểm 8.000đ + 8.000đ/km</li>
        <li style="color: #e74c3c;"><strong>⚠️ Phụ phí theo giờ Việt Nam:</strong></li>
        <ul>
            <li>🌙 <strong>Ban đêm (22:00 - 06:00 VN):</strong> +20%</li>
            <li>🌧️ <strong>Thời tiết mưa:</strong> +20%</li>
            <li>📈 <strong>Ví dụ:</strong> Nếu vào ban đêm + mưa = +40% tổng cộng</li>
            <li>🇻🇳 <strong>Thời gian hiện tại VN:</strong> {format_vietnam_time()}</li>
        </ul>
    </ul>
</div>
""")

# Main app container
app = widgets.VBox([
    header,
    vietnam_time_widget,  # Hiển thị thời gian Việt Nam ở đầu
    instructions,
    input_section,
    widgets.HTML("<hr style='border: 1px solid #bdc3c7; margin: 20px 0;'>"),
    result_output,
    vehicle_selection_output,
    map_output
], layout=widgets.Layout(padding='20px'))

print("✨ Ứng dụng đã sẵn sàng với thời gian Việt Nam chính xác!")
print(f"🇻🇳 Thời gian hiện tại tại Việt Nam: {format_vietnam_time()}")
print(f"🕐 Khung giờ: {get_time_period()}")
print(f"💰 Phụ phí ban đêm: {'CÓ (+20%)' if is_night_time() else 'KHÔNG'}")

# Tự động cập nhật thời gian mỗi khi chạy
update_vietnam_time_display()

display(app)

# Footer with updated info
display(HTML(f"""
<div class="result-box" style="text-align: center; background: linear-gradient(45deg, rgba(255,154,162,0.6) 0%, rgba(181,234,215,0.6) 100%); color: #000000; border: 2px solid rgba(255,255,255,0.4);">
    <h4 style="margin: 0; color: #000000;">🇻🇳 Chào mừng bạn đến với ứng dụng đặt xe thông minh!</h4>
    <p style="margin: 10px 0 0 0; color: #000000;">💰 Giá cả minh bạch | 🛣️ Tuyến đường tối ưu | ⭐ Đánh giá tài xế</p>
    <p style="margin: 5px 0 0 0; color: #000000;"><strong>🌙 Phụ phí theo giờ VN | 🌧️ Phụ phí thời tiết | 📊 Tính giá thông minh</strong></p>
    <p style="margin: 5px 0 0 0; color: #000000; font-size: 0.9em;"><strong>🇻🇳 Thời gian VN: {format_vietnam_time()} | {get_time_period()}</strong></p>
</div>
"""))

=== 🚀 KHỞI ĐỘNG ỨNG DỤNG ĐẶT XE THÔNG MINH VỚI THỜI GIAN VIỆT NAM ===
✨ Ứng dụng đã sẵn sàng với thời gian Việt Nam chính xác!
🇻🇳 Thời gian hiện tại tại Việt Nam: Thứ 6, 22/08/2025 - 09:40:28
🕐 Khung giờ: 🌅 Sáng
💰 Phụ phí ban đêm: KHÔNG
